In [1]:
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=57f8177e36aea59f9c9b7b4e2826e36490f49824a6282ecd38147ef7e3200160
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [2]:
# Librerías:
import datetime
import plotly.graph_objects as go
from statsmodels.tsa.api import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
from pandas_datareader import data as pdr
import pandas_datareader as pdr
from datetime import date
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.formula.api as sm
import pandas_ta as ta
import yfinance as yf
yf.pdr_override()
from datetime import datetime, timedelta
from plotly.subplots import make_subplots

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Cargar datos:
datos = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Trabajo investigación Bonos Colombianos/Rendimientos bonos mensual.xlsx")
#datos = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Trabajo investigación Bonos Colombianos/Rendimientos bonos diario .xlsx")
usdcop = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Trabajo investigación Bonos Colombianos/USDCOPdesde2003diario.xlsx")
tasasdeinteres = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Trabajo investigación Bonos Colombianos/Tasasdeinterescolombia.xlsx")
inflacion = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Trabajo investigación Bonos Colombianos/Inflacióncolombia.xlsx")
desempleo = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Trabajo investigación Bonos Colombianos/Desempleo.xlsx")
crecimiento = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Trabajo investigación Bonos Colombianos/Crecimiento anual.xlsx")

# ==============================================================================

# Ejecutar 1 vez la conversión de las fechas como índice:

datos["Fecha"]=pd.to_datetime((datos["Fecha"]))#, format='%Y%m')
#datos.set_index("Fecha", inplace=True)
#datos=datos.asfreq("M")
#]datos = datos["Fecha"].loc["2005-01-01":"2023-07-30"]

usdcop["Fecha"]=pd.to_datetime((usdcop["Fecha"]), format='%Y%m')
usdcop.set_index("Fecha", inplace=True)
usdcop = usdcop.asfreq("M")
#usdcop = usdcop.loc['2005-01-01':'2023-06-30']
usdcop = usdcop.reset_index()
#datos["usdcop"]=usdcop["Trm"]


tasasdeinteres["Fecha"]=pd.to_datetime((tasasdeinteres["Fecha"]), format='%Y%m')
tasasdeinteres.set_index("Fecha", inplace=True)
tasasdeinteres=tasasdeinteres.asfreq("M")
tasasdeinteres = tasasdeinteres.loc['2003-01-01':'2024-07-31']
tasasdeinteres = tasasdeinteres.reset_index()

desempleo["Fecha"]= pd.to_datetime((desempleo['Fecha']))#.astype(str), format='%Y%m')
desempleo.set_index("Fecha", inplace=True)
desempleo = desempleo.loc["2003-01-01":"2024-01-01"]
desempleo = desempleo.reset_index()

inflacion["Fecha"] = pd.to_datetime((inflacion['Fecha']).astype(str), format='%Y%m')
inflacion.set_index("Fecha", inplace=True)
inflacion = inflacion.loc['2003-01-01':'2024-07-01']
inflacion = inflacion.reset_index()
#inflacion["Fecha"] = datos["Fecha"]
#inflacion=inflacion.asfreq("M")

crecimiento["Fecha"] = pd.to_datetime((crecimiento["Fecha"]), format="%Y%m")
crecimiento.set_index("Fecha", inplace=True)
crecimiento = crecimiento.asfreq("M")
crecimiento = crecimiento.loc["2003-01-01":"2024-01-01"]
crecimiento = crecimiento.fillna(method='ffill', limit=100)
crecimiento = crecimiento.reset_index()

datos["Tasa"]=tasasdeinteres["Tasa"]
datos["Inflacion"]=inflacion["Inflación"]
datos["usdcop"]=usdcop["Trm"]
datos["Desempleo"]=desempleo["Desempleo %"]
datos["Crecimiento"]=crecimiento["Crecimiento anual %"]
datos = datos.fillna(method='ffill', limit=100)
datos.set_index("Fecha", inplace=True)
datos = datos.loc["2005-01-01":"2023-07-30"]
datos
# ==============================================================================

In [ ]:
datos.to_excel('Datos de bonos colombianos y variables macro.xlsx', index=True)

///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

Regresión líneal para explicar los datos de los rendimientos


In [ ]:
datos_corr = datos.corr()
datos_corr

In [ ]:
# Gráfico de la correlación
plt.figure(figsize=(12, 6))
sns.heatmap(datos_corr, annot=True, cmap="RdYlGn")
plt.title('Matriz de correlación de variables macroeconómicas')
#plt.savefig("correlación", dpi=1500)
plt.show()

In [7]:
# Modelo de regresión explicativo:
model = "Cinco~Tasa + Inflacion + Crecimiento"
results = sm.ols(formula=model, data=datos).fit()

In [8]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  Cinco   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.639
Method:                 Least Squares   F-statistic:                     131.2
Date:                Mon, 31 Jul 2023   Prob (F-statistic):           1.41e-48
Time:                        19:47:06   Log-Likelihood:                -377.02
No. Observations:                 222   AIC:                             762.0
Df Residuals:                     218   BIC:                             775.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       3.9318      0.212     18.512      

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Define the independent variables (features) and the dependent variable (target)
features = datos[['Tasa', 'Inflacion', 'Crecimiento']]
targets = datos[['Uno', 'Cinco', 'Diez']]

# Split the datos into training set and test set
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)

# Create a dictionary to store the models and their performance
models = {}
performance = {}

for bond in ['Uno', 'Cinco', 'Diez']:
    # Create a linear regression model
    model = LinearRegression()

    # Train the model
    model.fit(X_train, y_train[bond])

    # Make predictions
    predictions = model.predict(X_test)

    # Evaluate the model
    rmse = np.sqrt(mean_squared_error(y_test[bond], predictions))
    r2 = r2_score(y_test[bond], predictions)

    # Store the model and its performance
    models[bond] = model
    performance[bond] = {'RMSE': rmse, 'R^2': r2}

performance

{'Uno': {'RMSE': 0.6326463511976353, 'R^2': 0.9190763549386182},
 'Cinco': {'RMSE': 1.3243471494476127, 'R^2': 0.5948450956652576},
 'Diez': {'RMSE': 1.3425565526735155, 'R^2': 0.5120499313716524}}

///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [ ]:
# Gráfico interactivo de los rendimientos, las tasas y la inflación:

fig = go.Figure()
fig.add_trace(go.Scatter(x=datos.index, y=datos["Cinco"], name="Rendimiento bonos a 5 años"))
fig.add_trace(go.Scatter(x=datos.index, y=datos["Inflacion"], name="Inflacion"))
fig.add_trace(go.Scatter(x=datos.index, y=datos["Tasa"], name="Tasas de interés"))
#fig.add_trace(go.Scatter(x=datos.index, y=datos["usdcop"], name="Precio dólar"))
#fig.add_trace(go.Scatter(x=datos.index, y=datos["Desempleo"], name="Desempleo"))


fig.update_layout(
    title="Rendimiento bonos a 5 años",
    xaxis_title="Fecha",
    yaxis_title="Rendimiento",
    legend=dict(
        x=0,
        y=1,
        traceorder="normal",
        font=dict(size=12),
    )
)
fig.show()

In [ ]:
# Gráficos con diferentes escalas

# Crea una figura con especificaciones para un eje Y secundario
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Agrega las trazas al gráfico
fig.add_trace(go.Scatter(x=datos.index, y=datos["Cinco"], name="Rendimiento bonos a 5 años"), secondary_y=False)
fig.add_trace(go.Scatter(x=datos.index, y=datos["Inflacion"], name="Inflacion"), secondary_y=False)
fig.add_trace(go.Scatter(x=datos.index, y=datos["Tasa"], name="Tasas de interés"), secondary_y=False)
#fig.add_trace(go.Scatter(x=datos.index, y=datos["usdcop"], name="Precio dólar"), secondary_y=True)
fig.add_trace(go.Scatter(x=datos.index, y=datos["Crecimiento"], name="Crecimiento anual"), secondary_y=False)
#fig.add_trace(go.Scatter(x=datos.index, y=datos["Desempleo"], name="Desempleo"), secondary_y=False)

# Configura los títulos de los ejes y la leyenda
fig.update_layout(
    title="Rendimiento bonos a 5 años",
    xaxis_title="Fecha",
    yaxis_title="Rendimiento, Inflación, Tasas de interés",
    legend=dict(
        x=0,
        y=0,
        traceorder="normal",
        font=dict(size=12),
    )
)

# Configura el título del eje Y secundario
fig.update_yaxes(title_text="Precio dólar", secondary_y=True)

fig.show()

In [ ]:
# Crea una figura con 5 subgráficos
# fig = make_subplots(rows=3, cols=2)
#
# # Agrega las trazas al gráfico
# fig.add_trace(go.Scatter(x=datos.index, y=datos["Cinco"], name="Rendimiento bonos a 5 años"), row=1, col=1)
# fig.add_trace(go.Scatter(x=datos.index, y=datos["Inflacion"], name="Inflacion"), row=2, col=1)
# fig.add_trace(go.Scatter(x=datos.index, y=datos["Tasa"], name="Tasas de interés"), row=3, col=1)
# fig.add_trace(go.Scatter(x=datos.index, y=datos["usdcop"], name="Precio dólar"), row=1, col=2)
# fig.add_trace(go.Scatter(x=datos.index, y=datos["Crecimiento"], name="Crecimiento anual"), row=2, col=2)
# fig.add_trace(go.Scatter(x=datos.index, y=datos["Desempleo"], name="Desempleo"), row=3, col=2)
#
# # Configura los títulos de los ejes y la leyenda
# fig.update_layout(
#     title="Rendimiento bonos a 5 años, Inflación, Tasas de interés y Precio dólar",
#     xaxis_title="Fecha",
#     yaxis_title="Valor",
#     legend=dict(
#         x=0.5,
#         y=-0.1,
#         traceorder="normal",
#         font=dict(size=12),
#     )
# )
#
# fig.show()
#

///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

Uso del RSI

In [ ]:
# Aplicar Indicador Técnico RSI:
datos['RSI'] = ta.rsi(datos["Cinco"], length=5)
datos["Línea sobre compra"] = 70
datos["Línea sobre venta"] = 30
#datos = datos.set_index("Fecha")

In [ ]:
# Inicializa una variable para rastrear los cruces del RSI
cruce_rsi_compra = False
cruce_rsi_venta = False
cruce_indices_compra = []
cruce_indices_venta = []

# Recorre los datos del RSI
for i in range(1, len(datos["RSI"])):
    if datos["RSI"][i-1] > 70 and datos["RSI"][i] < 70:
      cruce_indices_compra.append(datos.index[i])
      cruce_rsi_compra = True

for i in range(1, len(datos["RSI"])):
    if datos["RSI"][i-1] < 15 and datos["RSI"][i] > 15:
      cruce_indices_venta.append(datos.index[i])
      cruce_rsi_venta = True

cruce_indices_compra=pd.DataFrame(cruce_indices_compra)
cruce_indices_venta=pd.DataFrame(cruce_indices_venta)

cruce_indices_compra.set_index([0], inplace=True)
cruce_indices_venta.set_index([0], inplace=True)

# Convertir los datos de las fechas
cruce_indices_compra=pd.to_datetime(cruce_indices_compra.index)
cruce_indices_venta=pd.to_datetime(cruce_indices_venta.index)

In [ ]:
# =============================================================================.
# Crea una figura con especificaciones para un eje Y secundario
fig = make_subplots(rows=2, cols=1)

# Gráfico de datos y RSI
fig.add_trace(go.Scatter(x=datos.index, y=datos["Cinco"], name="Datos originales"), row=1, col=1)
fig.add_trace(go.Scatter(x=datos.index, y=datos["RSI"], name="Valores del indicador", line=dict(color="blue")), row=2, col=1)
fig.add_trace(go.Scatter(x=datos.index, y=datos["Línea sobre compra"], name="Nivel sobre compra", line=dict(color='green')), row=2, col=1)
fig.add_trace(go.Scatter(x=datos.index, y=datos["Línea sobre venta"], name="Nivel sobre venta", line=dict(color='red')), row=2, col=1)

for fecha in cruce_indices_compra:
  fig.add_vline(x=fecha, line_dash="dash", line_color="green", line_width=2, opacity=0.8, name="Cruces RSI")

for fecha in cruce_indices_venta:
  fig.add_vline(x=fecha, line_dash="dash", line_color="red", line_width=2, opacity=0.8, name="Cruces RSI")

fig.update_layout(
    title="Estrategia aplicada al rendimiento de los bonos",
    xaxis_title="Fecha",
    yaxis_title="Valores",
    legend=dict(
        x=1,
        y=0,
        xanchor="auto",
        yanchor="auto",
        traceorder="normal",
        font=dict(size=12),
    )
)


fig.show()
# =============================================================================

In [ ]:
# =============================================================================
# Gráfico rendimientos con cruces
fig = go.Figure()
fig.add_trace(go.Scatter(x=datos.index, y=datos["Cinco"], name="Rendimiento de cierre mensual"))


for fecha in cruce_indices_compra:
  fig.add_vline(x=fecha, line_dash="dash", line_color="green", line_width=2, opacity=0.8, name="Cruces RSI")

for fecha in cruce_indices_venta:
  fig.add_vline(x=fecha, line_dash="dash", line_color="red", line_width=2, opacity=0.8, name="Cruces RSI")

fig.update_layout(
    title="Estrategia aplicada al rendimiento de los bonos a 5 años",
    xaxis_title="Fecha",
    yaxis_title="Rendimiento",
    legend=dict(
        x=0,
        y=1,
        traceorder="normal",
        font=dict(size=12),
    )
)
fig.show()
# =============================================================================

In [ ]:
# Gráfico RSI

# =============================================================================.
# Crea una figura con especificaciones para un eje Y secundario
fig = make_subplots(rows=1, cols=1)

# Gráfico de datos y RSI
#fig.add_trace(go.Scatter(x=datos.index, y=datos["Cinco"], name="Datos originales"), row=1, col=1)
fig.add_trace(go.Scatter(x=datos.index, y=datos["RSI"], name="Valores del indicador", line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=datos.index, y=datos["Línea sobre compra"], name="Nivel sobre compra", line=dict(color='green')), row=1, col=1)
fig.add_trace(go.Scatter(x=datos.index, y=datos["Línea sobre venta"], name="Nivel sobre venta", line=dict(color='red')), row=1, col=1)

#for fecha in cruce_indices_compra:
#  fig.add_vline(x=fecha, line_dash="dash", line_color="green", line_width=2, opacity=0.8, name="Cruces RSI")
#
#for fecha in cruce_indices_venta:
#  fig.add_vline(x=fecha, line_dash="dash", line_color="red", line_width=2, opacity=0.8, name="Cruces RSI")

fig.update_layout(
    title="Indicador de fuerza relativa",
    xaxis_title="Fecha",
    yaxis_title="Valores",
    legend=dict(
        x=1,
        y=0,
        xanchor="auto",
        yanchor="auto",
        traceorder="normal",
        font=dict(size=12),
    )
)


fig.show()
# =============================================================================


In [ ]:
# Sustentación con variables macroeconómicas:

# Gráficos con diferentes escalas

# Crea una figura con especificaciones para un eje Y secundario
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Agrega las trazas al gráfico
fig.add_trace(go.Scatter(x=datos.index, y=datos["Cinco"], name="Rendimiento bonos a 5 años"), secondary_y=False)
fig.add_trace(go.Scatter(x=datos.index, y=datos["Inflacion"], name="Inflacion"), secondary_y=False)
fig.add_trace(go.Scatter(x=datos.index, y=datos["Tasa"], name="Tasas de interés"), secondary_y=False)
#fig.add_trace(go.Scatter(x=datos.index, y=datos["usdcop"], name="Precio dólar"), secondary_y=True)
fig.add_trace(go.Scatter(x=datos.index, y=datos["Crecimiento"], name="Crecimiento anual"), secondary_y=False)
#fig.add_trace(go.Scatter(x=datos.index, y=datos["Desempleo"], name="Desempleo"), secondary_y=False)

for fecha in cruce_indices_compra:
  fig.add_vline(x=fecha, line_dash="dash", line_color="green", line_width=2, opacity=0.8, name="Cruces RSI")

for fecha in cruce_indices_venta:
  fig.add_vline(x=fecha, line_dash="dash", line_color="red", line_width=2, opacity=0.8, name="Cruces RSI")

# Configura los títulos de los ejes y la leyenda
fig.update_layout(
    title="Rendimiento de los bonos junto a variables macroeconómicas y la estrategia",
    xaxis_title="Fecha",
    yaxis_title="Valores",
    legend=dict(
        x=0,
        y=0,
        traceorder="normal",
        font=dict(size=12),
    )
)

# Configura el título del eje Y secundario
fig.update_yaxes(title_text="Precio dólar", secondary_y=True)

fig.show()